In [42]:
import os
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt

## *Data preprocessing*
### We cant feed videos to CNN model directly. Since a video is just a series of frames the approach will be:
#### 1) Loop over all frames in the video file
#### 2) For each frame, pass the frame through the CNN
#### 3) Obtain the predictions from the CNN
#### 4) Maintain a list of the last K predictions
#### 5) Compute the average of the last K predictions and choose the label with the largest corresponding probability
#### 6) Label the frame and write the output frame to disk

## *Converting videos into frames and writing them in a seprate folder*

In [43]:
data_path = 'C:/Users/ppras/ML/HRA/sample1/'

### Labeling different categories

In [46]:
categoeries = os.listdir(data_path)
labels = [i for i in range(len(categoeries))]
label_dict = dict(zip(categoeries, labels))

In [47]:
print(label_dict)
print(categoeries)
print(labels)

{'yo_yo': 0}
['yo_yo']
[0]


### For testing purpose we have short listed 10 activities the are:
#### 'Bowling', 'Boxing', 'Brushing_Teeth', 'Fencing', 'Handstand', 'tennis', 'typing', 'Walkng_witthDog', 'writing_on_board', 'yo_yo'

In [48]:
root_path = 'C:/Users/ppras/ML/HRA/frames/' # Path for writing frames from videos

### Reducing dimensions of pixels to (100x100)

In [30]:
down_width = 100
down_height = 100
down_points = (down_width,down_height)

In [51]:
img_count= 0
folder_count = 0
for items in categoeries: #looping over folders in categoeries
    path = os.path.join(root_path, items) 
    folder_path = os.path.join(data_path,items)
    img_names = os.listdir(folder_path) # Listing all files inside the directory
    os.mkdir(path)
    folder_count +=1
    
    for img_name in img_names:
            cap = cv2.VideoCapture(os.path.join(folder_path, img_name)) # Capcturing video and dividing it by frames
            success, image = cap.read()
            while success:
                success, image = cap.read()
                if not success:
                    break
                
                
                gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # Converting coloured image to greyScale
                norm = cv2.normalize(gray_image, None, alpha=0,beta=200, norm_type=cv2.NORM_MINMAX) # Normalize image
                resized_up = cv2.resize(norm, down_points, interpolation= cv2.INTER_LINEAR) # Resizing image
                cv2.imwrite(path + '/' + items +str(img_count) + '.jpg' , resized_up) # Writing image into to specified class with its class name 
                img_count += 1